## This is CNN model for NLP, tell the difference between data scientist and other engineer title

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline 

In [2]:
df1 = pd.read_csv('./data/data_data_scientist.csv')

In [3]:
df2 = pd.read_csv('./data/data_engineer.csv', nrows =25000)

In [4]:
df =pd.concat([df1, df2], sort =True)

In [5]:
df.reset_index(drop =True, inplace =True)

In [6]:
def find_data_scientist(title_value):
    target = 'data scientist'
    if type(title_value) == float:
        return -1
    elif type(title_value) ==str:
        if title_value.lower().find(target) >=0:
            return 1
        else:
            return 0

In [7]:
is_data_scientist =df.job_title.map(find_data_scientist)

In [8]:
is_data_scientist.unique()

array([ 1, -1,  0])

In [9]:
def find_data_scientist_binary(title_value):
    target = 'data scientist'
    if type(title_value) == float:
        return 0
    elif type(title_value) ==str:
        if title_value.lower().find(target) >=0:
            return 1
        else:
            return 0

In [10]:
is_data_scientist =df.job_title.map(find_data_scientist_binary)

### Add is_data_scientist column to indicate whether the title has data scientist

In [11]:
df['is_data_scientist']=is_data_scientist

In [12]:
df_new = df.dropna(subset =['job_description'])
df_new.reset_index(drop =True, inplace =True)

In [13]:
df_new.head()

,additional_information,job_ID,job_company,job_date,job_description,job_title,person_ID,skills,is_data_scientist
0,Core CompetenciesData Modeling & Visualization...,1,"TRILLO INC - San Francisco Bay Area, CA",January 2018 to Present,* Incorporate Machine Learning solutions to fa...,Data Scientist,1,"CLUSTERING (Less than 1 year), DATA MINING (Le...",1
1,TECHNICAL EXPERTISEStatistics/Machine Learning...,1,"Pepsico - Plano, TX",September 2018 to Present,Fleet Analysis: Exploring the Truck repair and...,Data Scientist,2,"Hadoop (Less than 1 year), PYTHON (Less than 1...",1
2,TECHNICAL EXPERTISEStatistics/Machine Learning...,2,"CISCO - San Jose, CA",May 2018 to July 2018,Developed Python scripts to automate the load...,Data Scientist,2,"Hadoop (Less than 1 year), PYTHON (Less than 1...",1
3,TECHNICAL EXPERTISEStatistics/Machine Learning...,3,"PHOTON INFOTECH - Bohemia, NY",September 2017 to March 2018,Developed a personalized recommender engine fo...,Data Scientist,2,"Hadoop (Less than 1 year), PYTHON (Less than 1...",1
4,TECHNICAL EXPERTISEStatistics/Machine Learning...,4,"APPLE INC - Cupertino, CA",January 2016 to August 2017,Played a key role in developing and maintainin...,Data Scientist,2,"Hadoop (Less than 1 year), PYTHON (Less than 1...",1


In [14]:
print(df_new.shape)

(46773, 9)


In [15]:
sum(df_new.is_data_scientist ==1)

6321

In [16]:
import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
#from keras.layers import LSTM, Bidirectional
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
import gensim
from keras.callbacks import Callback

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import codecs
import matplotlib.pyplot as plt

Using TensorFlow backend.


### Below code is a test for concat function

In [ ]:
inp1 = Input(shape=(10,20))
inp2 = Input(shape=(10,32))
cc1 = concatenate([inp1, inp2], axis=2) # Merge column, same row
output = Dense(30, activation='relu')(cc1)

In [ ]:
model = Model(inputs=[inp1, inp2], outputs=output)

In [ ]:
model.summary()

In [ ]:
print(model.layers[0].output)

In [ ]:
t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]

In [ ]:
inp1 = Input(shape=(2,3))
inp2 = Input(shape=(2,3))
cc1 = concatenate([inp1, inp2], axis=2) # Merge column, same row
output = Dense(30, activation='relu')(cc1)

In [ ]:
model = Model(inputs=[inp1, inp2], outputs=output)

### The code above is for testing

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))

In [ ]:
EMBEDDING_DIM = 300 # how big is each word vector
MAX_VOCAB_SIZE = 150000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 200 # max number of words in a comment to use

#training params
batch_size = 256 
num_epochs = 2 

In [ ]:
temp = df_new['job_description'].map(lambda x: x.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " "))

In [ ]:
temp[0]

In [ ]:
len(temp)

In [24]:
##define a class, which has fit, transform and fit_transform function
class Convert_pad_sequence(object):
    def __init__(self):
        self.EMBEDDING_DIM = 300 # how big is each word vector
        self.MAX_VOCAB_SIZE = 100000 # how many unique words to use (i.e num rows in embedding vector)
        self.MAX_SEQUENCE_LENGTH = 200 # max number of words in a comment to use

        #training params
        #self.batch_size = 256 
        #self.num_epochs = 2 
        
        #load word2vect dictionary
        word2vec_path = "./data/GoogleNews-vectors-negative300.bin.gz"
        self.word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, \
                                                                        binary=True, limit=300000)
    
    def fit(self, input_data_series, y =None):
        if y.size:
            array_size = y.shape
            if len(array_size)==1:
                self.labels_index =1
            else:
                self.labels_index = array_size[1]
                
        self.tokenizer =Tokenizer(num_words=self.MAX_VOCAB_SIZE, lower=True, char_level=False)
        self.tokenizer.fit_on_texts(input_data_series.tolist())
        train_word_index = self.tokenizer.word_index
        self.len_index = len(train_word_index)
        self.train_embedding_weights = np.zeros((self.len_index+1, self.EMBEDDING_DIM))
        
        #training_sequences = self.tokenizer.texts_to_sequences(input_data_frame["job_description"].tolist())
        self.train_embedding_weights = np.zeros((self.len_index+1, self.EMBEDDING_DIM))
        #train_embedding_weights = np.zeros((MAX_VOCAB_SIZE+1, EMBEDDING_DIM))
        for word,index in train_word_index.items():
            self.train_embedding_weights[index,:] = self.word2vec[word] if \
                                               word in self.word2vec else np.random.rand(self.EMBEDDING_DIM)
            
    def transform(self, input_data_series, y =None):
        training_sequences = self.tokenizer.texts_to_sequences(input_data_series.tolist())
        train_cnn_data = pad_sequences(training_sequences, maxlen=self.MAX_SEQUENCE_LENGTH)
        
        #return a tuple
        return (train_cnn_data, self.train_embedding_weights, 
                self.MAX_SEQUENCE_LENGTH, self.len_index+1,
                self.EMBEDDING_DIM, self.labels_index)
    
    
    
    def fit_transform(self, input_data_series, y=None):
        self.fit(input_data_series, y)
        
        return self.transform(input_data_series)
        
        
        

In [18]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [19]:
class CNN_text(object):
    def __init__(self):
        pass
    
    def _ConvNet(self,embeddings, max_sequence_length, num_words, embedding_dim, \
                labels_index, trainable=False, extra_conv=True):
    
        embedding_layer = Embedding(num_words,
                                   embedding_dim,
                                   weights=[embeddings],
                                   input_length=max_sequence_length,
                                   trainable=trainable)

        sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
        embedded_sequences = embedding_layer(sequence_input)

        # Yoon Kim model (https://arxiv.org/abs/1408.5882)
        convs = []
        filter_sizes = [3,4,5]

        for filter_size in filter_sizes:
            l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
            l_pool = MaxPooling1D(pool_size=3)(l_conv)
            convs.append(l_pool)

        #l_merge = Merge(mode='concat', concat_axis=1)(convs)
        l_merge = concatenate(convs, axis=1)

        # add a 1D convnet with global maxpooling, instead of Yoon Kim model
        conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
        pool = MaxPooling1D(pool_size=3)(conv)

        if extra_conv==True:
            x = Dropout(0.5)(l_merge)  
        else:
            # Original Yoon Kim model
            x = Dropout(0.5)(pool)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.5)(x)
        # Finally, we feed the output into a Sigmoid layer.
        # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
        # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
        preds = Dense(labels_index, activation='sigmoid')(x)

        model = Model(sequence_input, preds)
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['acc'])
        model.summary()
        return model
    
    def fit(self, input_data, y=None):
        x_train = input_data[0]
        y_train = y.values
        X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
        embeddings = input_data[1]
        max_sequence_length = input_data[2]
        num_words = input_data[3]
        embedding_dim = input_data[4]
        labels_index = input_data[5]
        
        batch_size = 256
        num_epochs = 2
        
        self.model = self._ConvNet(embeddings, max_sequence_length, num_words, 
                                   embedding_dim, labels_index, False)
        
        self.model.fit(X_tra, y_tra, batch_size=batch_size, epochs=num_epochs, 
                       validation_data=(X_val, y_val),
                       callbacks=[RocAuc], verbose=2)
        
    def transform(self, input_data, y=None):
        pass
    
    def fit_transform(self, input_data, y=None):
        self.fit(input_data, y)
        self.transform()
        
    def predict(self, input_data, batch_size=None, verbose=0, steps=None):
        return self.model.predict(input_data[0], batch_size, verbose, steps)
        

In [20]:
def standardize_text(df, text_field, job_label):
    text_first =df[text_field].map(lambda x: x.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " "))
    text_second = text_first.map(lambda x: x.lower())
    #df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    #df[text_field] = df[text_field].str.lower()
    df_copy = pd.concat([text_second, df[job_label]], axis =1)
    return df_copy

In [21]:
train_comments =standardize_text(df_new, 'job_description', 'is_data_scientist')

In [22]:
type(train_comments)

pandas.core.frame.DataFrame

In [23]:
train_comments.head()

,job_description,is_data_scientist
0,* incorporate machine learning solutions to fa...,1
1,fleet analysis: exploring the truck repair and...,1
2,developed python scripts to automate the load...,1
3,developed a personalized recommender engine fo...,1
4,played a key role in developing and maintainin...,1


In [ ]:
#MAX_VOCAB_SIZE = 94918

In [ ]:
#test_input = C_pad_sequence.transform(test_series)

In [ ]:
#prob = cnn_text_model.predict(test_input[0])
#print('prob is: ',prob[0][0] )

In [25]:
steps =[('convert_to_pad', Convert_pad_sequence()), 
        ('cnn_text_model', CNN_text())]
model = Pipeline(steps)

In [26]:
model.fit(train_comments['job_description'], train_comments['is_data_scientist'])

/Users/fengyi/.pyenv/versions/3.6.4/envs/try_deep_learning/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 300)     31821300    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 198, 128)     115328      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 197, 128)     153728      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

Pipeline(memory=None,
     steps=[('convert_to_pad', <__main__.Convert_pad_sequence object at 0x135b40438>), ('cnn_text_model', <__main__.CNN_text object at 0x135b40c50>)])

In [33]:
test_input_string = 'modeling for prediction, use python for programming, linear regression, classification,\
                     collect data, explore, random forest '
test_series = pd.Series([test_input_string])

In [34]:
prob = model.predict(test_series)
print('prob is: ', prob[0][0])

prob is:  0.53039086


In [36]:
print(model.steps[0][0])

convert_to_pad


In [37]:
print(model.steps[0][1])

In [39]:
print(model.steps[1][1])

In [41]:
import pickle

In [42]:
with open('./data/cnn_data_scientist_preprocsssing.pkl', 'wb') as f:
    pickle.dump(model.steps[0][1],f)

In [43]:
model.steps[1][1].model.save('./data/cnn_data_scientist_modeling.h5')

### Below different callback mode

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

hist = model.fit(x_train, y_tr, epochs=num_epochs, callbacks=callbacks_list, 
                 validation_split=0.1, shuffle=True, batch_size=batch_size)

### Above different callback mode

In [ ]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=num_epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=2)

In [ ]:
model.save('./data/cnn_text_model.h5')

In [ ]:
#generate plots
plt.figure()
plt.plot(hist.history['loss'], lw=2.0, color='b', label='train')
plt.plot(hist.history['val_loss'], lw=2.0, color='r', label='val')
plt.title('CNN sentiment')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy Loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.figure()
plt.plot(hist.history['acc'], lw=2.0, color='b', label='train')
plt.plot(hist.history['val_acc'], lw=2.0, color='r', label='val')
plt.title('CNN sentiment')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()